In [2]:
!pip install --quiet transformers
!pip install --quiet datasets
!pip install --quiet transformers[torch]
!pip install --quiet accelerate -U
!pip install --quiet matplotlib
!pip install --quiet seaborn
!pip install --quiet -U scikit-learn
!pip install --quiet nltk
!pip install --quiet wandb
!pip install pyarrow
!pip install --upgrade pyarrow

In [3]:
from collections import defaultdict
from tqdm import tqdm
import requests
import torch

from transformers import pipeline
from transformers import GPT2Tokenizer, AutoTokenizer
from datasets import load_dataset, Dataset, DatasetDict
from transformers import GPT2LMHeadModel, AutoConfig
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F

In [4]:
import os
import s3fs
import json

BUCKET_OUT = "mamazo"  

S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
fs.ls(BUCKET_OUT)

['mamazo/StatApp']

In [5]:
#with fs.open(LOGIN+"/StatApp/tokenized_128_top10.json", 'r') as file:
#   tokenized_test = Dataset.from_dict({"input_ids": json.load(file)})

# Entropy ?

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

#import nltk
import nltk
from nltk.corpus import stopwords

#stop-words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
#Ce code est beaucoup trop sous-optimisé et ne sert pas à grand chose


def spectral_entropy(matrix):
    eigenvalues, _ = np.linalg.eig(matrix)
    
    entr = 0
    for val in eigenvalues:
        if val > 0 :
            entr += val*np.log(val)
    
    return np.exp(-entr)

def maximize_entropy(corpus, threshold_entr = np.inf, max_features=5000, seed_value=345, size_selected=10, n_components = None, affiche_index = False):

    n = len(corpus["text"])

    if size_selected > n:
        size_selected = n
    if n_components == None:
        n_components = round((size_selected)**(0.5))

    np.random.seed(seed_value)

    #Sélection du premier texte du corpus
    selected_document = np.random.choice(corpus["text"])
    index = corpus["text"].index(selected_document)

    #Sélection du deuxième texte du corpus

    if index == n-1:
        index2 = index-1
    else :
        index2 = index+1
    selected_indexes = [index,index2]
    
    #Création du sous-corpus dans lequel on va ajouter les documents que l'on va sélectionner
    selected_text = corpus['text'][index]
    selected_text2 = corpus['text'][index2]
    sub_corpus = DatasetDict({"text": [selected_text,selected_text2]})

    #Création du vectorizer
    stop_words = nltk.corpus.stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=max_features)

    entr_tot = 0
    for k in range(size_selected-2):
        
        #Création d'un sous-corpus temporaire
        sub_corpus2 = sub_corpus

        #On ne sélectionne des textes que si on a pas déjà "suffisamment d'entropie".
        if entr_tot < threshold_entr: 
            
            max_entr_tot = 0
            index = 0
            for j in range(0,n-1):
                
                #On trouve un texte que l'on n'a pas déjà utilisé
                if j not in selected_indexes:

                    #Ajout du texte au corpus temporaire
                    selected_text = corpus['text'][j]
                    sub_corpus2["text"].append(selected_text)

                    #Calcul de la matrice de similarité et de son entropie spectrale
                    tfidf_matrix = vectorizer.fit_transform(sub_corpus2)
                    lsa_model = TruncatedSVD(n_components=n_components, n_iter=10, random_state=seed_value)
                    lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
                    similarity_matrix = cosine_similarity(lsa_matrix, lsa_matrix)
                    new_entr = spectral_entropy(similarity_matrix)
                    
                    if new_entr > max_entr_tot:
                        index = j
            
            #On ajoute le texte maximisant l'entropie pour un nombre k+2 de textes du sous-corpus
            selected_text = corpus['text'][index]
            sub_corpus["text"].append(selected_text)
            selected_indexes +=[index]
            entr_tot = max_entr_tot

    return sub_corpus, entr #On renvoie le sous-corpus ainsi que l'entropie du susnommé


In [12]:
def select_least_similar(corpus, max_features=5000, seed_value=345, size_selected=100, n_components=None, affiche_index=False, max_df=0.5, min_df=3):

    if n_components is None:
        n_components = round((size_selected) ** 0.5)

    selected_documents = []
    selected_indices = []

    stop_words = nltk.corpus.stopwords.words('english')

    stop_words = nltk.corpus.stopwords.words('english')
    vectorizer = TfidfVectorizer(stop_words=stop_words, max_features=max_features, max_df=max_df, min_df=min_df)

    tfidf_matrix = vectorizer.fit_transform(corpus)
    lsa_model = TruncatedSVD(n_components=n_components, n_iter=10, random_state=seed_value)
    lsa_matrix = lsa_model.fit_transform(tfidf_matrix)
    similarity_matrix = cosine_similarity(lsa_matrix, lsa_matrix)
    
    eig_val,_ = np.linalg.eig(similarity_matrix)
    gini_val = np.power(eig_val,-eig_val)

    sel,selected_indices = select_n_highest(gini_val,size_selected)

    if affiche_index == True:
        print(selected_indices)

    return data.select(selected_indices)

def select_n_highest(array, n):
    sorted_indices = np.argsort(-array)
    selected_indices = sorted_indices[:n]
    selected_indices.sort()
    selected_values = array[selected_indices]
    return selected_values, selected_indices


In [9]:

BUCKET_OUT = "mamazo"
file='top_500_sur_10000.json'

CORPUS_S3 = BUCKET_OUT + "/" + "StatApp/" + file
with fs.open(CORPUS_S3, 'r') as f:
        data = Dataset.from_list(json.load(f))


In [13]:
X = Dataset.from_dict(data[:1000])

print(select_least_similar(X["text"], size_selected= 900, affiche_index=True))

[ 31  32  33  34  35  36  37  38  39  40  42  43  44  45  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76 101 102 103 104 105 106 107 108 109 110 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 142 143 144 145 146 147 148 149 156 157 158 159 166 167 173 174 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 216 217
 218 219 224 225 226 227 228 231 232 235 236 237 238 239 240 241 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279
 280 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297
 298 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315
 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333
 334 335 336 337 338 339 340 341 342 343 344 345 34

In [11]:
        #else:
            
        #    index = np.randint(0,n-1)
        #    while index in selected_indexes:
        #        index = np.randint(0,n-1)
        #    selected_indexes += [index]
        #    selected_text = corpus['text'][index]
        #    sub_corpus.append({'text': selected_text})
        #    entr_tot = max_entr_tot